#### [Weights and Biases](https://www.wandb.com/) is a tool for visualizing and tracking machine learning experiments.

In [ ]:
# Import abilities
from ai import Chat, Vision, Audio, Images, Embeddings, FineTuner
import powers
import wandb
import time
from datetime import datetime
import wandb_logging as wb

# Adjust model globally here or modify in function calls. Vision model cannot be changed, for now ;) 
model = "gpt-3.5-turbo-1106"

In [37]:
def log_all_spans(span, name):
    # Log the current span
    span.log(name=name)

    for attribute, value in vars(span._span).items():
        if attribute == "child_spans":
            print(f"{attribute}: {len(value)}")
            for child_span in value:
                print(f"    {child_span}")

    # Access the child_spans attribute of the span object, if it exists
    child_spans = getattr(span._span, 'child_spans', [])
    
    # Check if child_spans is not empty
    if child_spans:
        # Iterate over each child span
        for child_span in child_spans:
            # Construct the Trace object with _model_dict set to None and _span set to the child_span object
            trace = span
            # Log the Trace object
            trace.log(name=name)

## figure out how to construct the trace object for the nested spans
## figure out how to log the trace object
## we want a single log_all_spans function for the end of an application run to log everything to wandb.
## could handle the multiagent case by having a method which updates the end time of the child spans in the chain leading to the current span
## is it possible to have it such that the user only has to include the parent span, and all the orchestration is automatically handled? Perhaps using metadata?

## Adding images


### Weights and Biases x OpenAI-Launchpad

Using wandb prompts and traces to visualize the flow in AI systems.

In [ ]:
audio = Audio()
chat = Chat(model=model, system="Helpful assistant.")

wandb.init(project="openai-launchpad") ## Initialize wandb project

chain_name = "conversation"

chain_span , _ , _ = wb.wandb_span(span_kind = "agent", span_name=chain_name, parent_span=None, root_span=None)

for i in range(3):

    # transcript = audio.record_and_transcribe()
    transcript = "This is a test."
    time.sleep(1)

    transcript_span, _ , chain_span = wb.wandb_span(
        span_kind="tool",
        span_name="transcription",
        inputs={"audio": "audio"},
        outputs={"transcript": transcript},
        parent_span=None,
        root_span=chain_span,
    )

    if "quit" in transcript:
        break

    completion, messages = chat.chat_completion(transcript, speak=False, stream=True, memories=True)

    # Log the LLM used.
    llm_span, _ ,  chain_span = wb.wandb_span(
        span_kind="llm",
        span_name="chat",
        inputs={"transcript": transcript},
        outputs={"completion": completion, "messages": messages},
        parent_span = None,
        root_span=chain_span,
    )

    voice = "echo"
    # audio.speak(completion, voice=voice)
    time.sleep(2)

    # Log the speech span.
    speech_span, llm_span , chain_span = wb.wandb_span(
        span_kind="tool",
        span_name="speech",
        inputs={"completion": completion},
        outputs={"tool": "Completion spoken."},
        parent_span=llm_span,
        root_span=chain_span,
        metadata={"voice": voice}
    )

conversation_end_time_ms = round(datetime.now().timestamp() * 1000)
chain_span.end_time_ms = conversation_end_time_ms

# log_all_spans(chain_span, chain_name)

chain_span.log(name=chain_name)

# End wandb run
wandb.finish()

In [ ]:
for attribute, value in vars(chain_span._span).items():
    if attribute == "child_spans":
        print(f"{attribute}: {len(value)}")
        for child_span in value:
            print(f"    {child_span}")

In [ ]:
# Initialize wandb project
wandb.init(project="openai-launchpad")

# Define the chain name for logging
chain_name = "image_generation_and_vision"

image_prompt = "The power of time in the palm of your hand."

# Start the chain for logging
chain_span, _ , _ = wb.wandb_span(span_kind="chain", span_name=chain_name, parent_span=None, root_span=None)

# Generate an image
images = Images()
response, path = images.generate_image(image_prompt, display_image=True, save_image=True)

# Log the image generation span
image_gen_span, _ , chain_span = wb.wandb_span(
    span_kind="tool",
    span_name="image_generation",
    inputs={"prompt": image_prompt},
    outputs={"response": response, "image_path": path},
    parent_span=None,
    root_span=chain_span,
    metadata={"display_image": True, "save_image": True}
)

# Vision system analysis
vision = Vision(system="You can only respond with bullet points.")
response, messages = vision.vision_completion(prompt="What is this image showing?", image_paths=[path], memories=True, stream=True)

# Log the vision system span
vision_span, _ , chain_span = wb.wandb_span(
    span_kind="llm",
    span_name="vision_description",
    inputs={"prompt": "What is this image showing?", "image_paths": [path]},
    outputs={"response": response, "messages": messages},
    parent_span=None,
    root_span=chain_span
)

chain_span.log(name=chain_name)

# End wandb run
wandb.finish()